In [ ]:
%reset

In [ ]:
!pip install pymongo

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
uri = ("YOUR_URL")

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
from experiments_class import Experiments, DatabaseExperiments, Conversation, Message
from transformers import pipeline

db = client['final_experiments']
conversations = db['conversations']
messages = db['messages']
experiments = db['experiments']

mess_dict = {}

for mess in messages.find():
    mess_dict[mess['_id']] = Message(mess)

toxicity_model = pipeline(model='tomh/toxigen_roberta', tokenizer='tomh/toxigen_roberta', batch_size=32, device=0, truncation='longest_first')


keys = []
content = []
    
for k in mess_dict.keys():
    keys.append(k)
    content.append(mess_dict[k].content)

out = toxicity_model(content)

for idx in range(len(out)):
  if out[idx]['label'] == 'LABEL_0':
      mess_dict[keys[idx]].toxicity_score = 1.0 - out[idx]['score']
  else:
      mess_dict[keys[idx]].toxicity_score = out[idx]['score']    
      
conv_dict = {}

for conv in conversations.find():
    conv_dict[conv['_id']] = Conversation(conv, mess_dict)

exp_list = []

for exp in experiments.find():
    exp_list.append(Experiments(exp, conv_dict))
    
db_exp = DatabaseExperiments(exp_list)

In [ ]:
print(len(mess_dict.keys()))

In [ ]:
print(len(conv_dict.keys()))

In [ ]:
models = ["mistral:latest",
          "llama3:latest",
          "command-r:latest",
          "mixtral:latest",
          "orca2:latest"]

creator = ["nicolo",
           "admin",
           "gian_maria"]

goal = ["Escape",
        "Yard Time"]

personality_prisoner = ["Good Prisoner",
                        "Bad Prisoner"]

personality_guard = ["Good Guard",
                        "Bad Guard"]

neutral =  "Blank Personalities"

sections = ['Communication rules', 'Environment', 'Research oversight', 'Risks']

In [ ]:
print(str(models[3]))

In [ ]:
sample_conv = db_exp.filter_conversations(llm=models[2], shared_sections=[sections[0], sections[1], sections[3]], goal=goal[0], personality_prisoner=personality_prisoner[1], personality_guard=personality_guard[1])


In [ ]:
print(len(db_exp))

In [ ]:
print(len(sample_conv))

In [ ]:
print(sample_conv[14].print_conversation())

# ANALYSIS

In [ ]:
conv = sample_conv[6]

conv.print_conversation()

In [ ]:
def analyse_comments(comments, model):
  
  out = toxicity_model(comments)
  tox_score = list()

  for idx in range(len(sent)):
    if out[idx]['label'] == 'LABEL_0':
      tox_score.append(1.0 - out[idx]['score'])
    else:
      tox_score.append(out[idx]['score'])

  return(tox_score)

def analyse_conversation(conversation):
  all_message = []
  pris_message = [] 
  guard_message = []
  for mess in conversation.messages:
    all_message.append(mess.toxicity_score)
    if mess.role == "Prisoner":
      all_message.append(mess.toxicity_score)
      pris_message.append(mess.toxicity_score)
    else:
      if mess.role == "Guard":
        all_message.append(mess.toxicity_score)
        gurad_message.append(mess.toxicity_score)
  return all_message, pris_message_guard_message
  

In [ ]:
import pickle

with open('database_exp.pkl', 'wb') as f:
    pickle.dump(db_exp, f)